# Kryptologie LAB - CBC

In [135]:
import numpy as np
import random

## AES

In [136]:
# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [137]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox")
    print(sbox)

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox Inverse")
    print(sbox_inverse)

def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")

def blockToText(textblock):
    text = textblock.transpose().flatten()
    toChar = lambda c: chr(c)
    return "".join(np.array([toChar(c) for c in text]))

def addRoundKey(textblock, key):
    chars = [ord(c) for c in key]
    key = np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")
    return np.bitwise_xor(textblock, key)

def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            row = (textblock[y,x] >> 4) & 0xF
            col = textblock[y,x] & 0xF
            textblock[y,x] = box[row, col]
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

def encryptBlock(textblock, keys, inputAsMatrix=False, returnAsBlock=False):
    if not inputAsMatrix:
        textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    if returnAsBlock:
        return textblock
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

SBox
[[ 99 124 119 123 242 107 111 197  48   1 103  43 254 215 171 118]
 [202 130 201 125 250  89  71 240 173 212 162 175 156 164 114 192]
 [183 253 147  38  54  63 247 204  52 165 229 241 113 216  49  21]
 [  4 199  35 195  24 150   5 154   7  18 128 226 235  39 178 117]
 [  9 131  44  26  27 110  90 160  82  59 214 179  41 227  47 132]
 [ 83 209   0 237  32 252 177  91 106 203 190  57  74  76  88 207]
 [208 239 170 251  67  77  51 133  69 249   2 127  80  60 159 168]
 [ 81 163  64 143 146 157  56 245 188 182 218  33  16 255 243 210]
 [205  12  19 236  95 151  68  23 196 167 126  61 100  93  25 115]
 [ 96 129  79 220  34  42 144 136  70 238 184  20 222  94  11 219]
 [224  50  58  10  73   6  36  92 194 211 172  98 145 149 228 121]
 [231 200  55 109 141 213  78 169 108  86 244 234 101 122 174   8]
 [186 120  37  46  28 166 180 198 232 221 116  31  75 189 139 138]
 [112  62 181 102  72   3 246  14  97  53  87 185 134 193  29 158]
 [225 248 152  17 105 217 142 148 155  30 135 233 206  85

## CBC

In [138]:
blockLength = 16
IV = np.zeros((4,4), dtype=np.dtype('B'))
key = ["EINSCHLUESSEL123" for i in range(11)]

In [139]:
def getChunks(text, blockLength=16):
    return [text[i:i+blockLength] for i in range(0, len(text), blockLength)]

def CBCMAC(k, chunks):
    xorVec = IV
    output = ""
    for chunk in chunks:
        chunkAsBytes = initBlock(chunk)
        inputBytes = np.bitwise_xor(chunkAsBytes, xorVec)
        xorVec = encryptBlock(inputBytes, k, inputAsMatrix=True, returnAsBlock=True)
        output = xorVec
    return blockToText(output)

In [140]:
text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
chunks = getChunks(text, blockLength)
print("Chunks:\n")
print(chunks)
CBCMAC(key, chunks)

Chunks:

['Lorem ipsum dolo', 'r sit amet, cons', 'etetur sadipscin', 'g elitr, sed dia', 'm nonumy eirmod ', 'tempor invidunt ', 'ut labore et dol', 'ore magna aliquy', 'am erat, sed dia', 'm voluptua. At v', 'ero eos et accus', 'am et justo duo ', 'dolores et ea re', 'bum. Stet clita ', 'kasd gubergren, ', 'no sea takimata ', 'sanctus est Lore', 'm ipsum dolor si', 't amet. Lorem ip', 'sum dolor sit am', 'et, consetetur s', 'adipscing elitr,', ' sed diam nonumy', ' eirmod tempor i', 'nvidunt ut labor', 'e et dolore magn', 'a aliquyam erat,', ' sed diam volupt', 'ua. At vero eos ', 'et accusam et ju', 'sto duo dolores ', 'et ea rebum. Ste', 't clita kasd gub', 'ergren, no sea t', 'akimata sanctus ', 'est Lorem ipsum ', 'dolor sit amet.']


'A{¢¯á\x81\x10k%ÿ\x06(©D\x9cì'

## CCM-Modus

In [141]:
def initBlockFromInt(n: int):
    assert 0 <= n < 2 ** 128
    bytes = []
    for byte in range(15,-1,-1):
        bytes.append((n >> (8 * byte)) & 0xFF)
    return np.array(bytes, dtype=np.uint8).reshape((4,4)).transpose()

def counter(k, nonce, inputChunks):
    output = ""
    init_ctr = nonce << 64
    # Counter encryption
    for i in range(len(inputChunks)):
        chunkAsBytes = initBlock(inputChunks[i])
        T_i = (init_ctr + i) % (2 ^ 128)
        output += blockToText(np.bitwise_xor(chunkAsBytes, encryptBlock(initBlockFromInt(T_i), k, inputAsMatrix=True, returnAsBlock=True)))
    return output

def encryptAndSignCCM(k, nonce, inputChunks):
    output = counter(k, nonce, inputChunks)
    # CBCMac
    #hash = CBCMAC(k, chunks)
    #output += hash
    return output

def decryptAndVerify(k, nonce, inputChunks):
    message = counter(k, nonce, inputChunks)
    print("Message:\n", message)
    # CBCMac
    #hash = CBCMAC(k, getChunks(message))
    #expectedHash =  chunks[-1]
    #print("exctracted: ",expectedHash)
    #assert hash == expectedHash
    return message

In [149]:
text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
chunks = getChunks(text, blockLength)
nonce = random.getrandbits(64)
y = encryptAndSignCCM(key, nonce, chunks)
print("CCM:\n",y)
message = decryptAndVerify(key, nonce, getChunks(y))

CCM:
 óº2ð2s¥Å]^M«ë(L_>0÷
¾Æt,t­;Öy´"¦©ç8æÙõ÷á¡|ùóvkSÓ-KÚ"Ü?ÎW_JSßp¯7¨Ë(bËºGN&äñv{â]'ÔÓÏ3<'à±ã5O"9ujÏWK¦fõ(Lb+4vü¶ä!¸»JB¨­qÜVéMgñ4Ï²ßÙ~ãß½è¦ìø,ÕË¯õYýáy1+D+IYµ]0iæâ»Ì[Ý°ânÖHÆ¨±KCß¸ñCRY÷*µ­ÀµåY··1õ}Wl©áhJ²]ëýÄXaÂ,7 áñ #òki¸¬Iiò<KÞ£í¾oÑïZü¥ºS¨MþìH2tå\?ðÓÝ5Ñ¥©¯È¨²GÄãÈìGw5à	¿BôÂÊ?º»	2E:ÿII·Hîv_õ&ôwÕÂÐüÆ"[iÜ.?é³_Td8.ÁôDÈ®`Ê¢KqëaÖ®Ó-ØºJã=Ñã6"Íµ	~¥64²Û-ÓÌj(âÆ@ú*y¡²s¿þ!Y(ûÖ]3§~g5¤HD%,7·OV»ðv~?üMpÇ ·b'»·¸Å¾!
ª-p¸2Ê7
]ÐÝoäÍ´,iDAqCXàÓ2J¯ÙayÔ¤uñüõ7?¼iØk@{
Message:
 Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumyi)ÛË	Þ¶¿óÍ58!|ZS¶CN-cÁÒ}·